In [1]:
import psycopg2
import pandas as pd

username = "postgres"
password = "test"
hostname = "db"
port = "5432"
database = "test"

dst = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"
con = psycopg2.connect(dst)

In [11]:
# create table
def create_shohin_table(cur):
    table_name = 'shohin'
    cur.execute(f"drop table if exists {table_name}")
    cur.execute(f"""create table {table_name} (
        shohin_id char(4) not null,
        shohin_mei varchar(100) not null,
        shohin_bunrui varchar(32) not null,
        hanbai_tanka integer,
        shiire_tanka integer,
        torokubi date,
        primary key (shohin_id));""")
    return cur

with psycopg2.connect(dst) as con, con.cursor() as cur:
    try:
        create_shohin_table(cur)
        sql = f"select * from shohin"
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as er:
        print('psycopg2.Error:', er)

Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []


In [19]:
# drop table
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        cur.execute(f"drop table {table_name};")
        sql = f"select * from shohin"
        print(pd.read_sql(sql, con=con))
    except pd.io.sql.DatabaseError as e:
        print(f"expected error {e}") 

expected error Execution failed on sql 'select * from shohin': relation "shohin" does not exist
LINE 1: select * from shohin
                      ^



In [24]:
# arrange column
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        create_shohin_table(cur)
        
        sql = f"select * from {table_name};"
        print(pd.read_sql(sql, con=con))
        
        cur.execute(f"alter table {table_name} add column shohin_mai_kana varchar(100)")
        print(pd.read_sql(sql, con=con))
        
        cur.execute(f"alter table {table_name} drop column shohin_mai_kana;")
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []
Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi, shohin_mai_kana]
Index: []
Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []
